Importing Libraries

In [1]:
import pandas as pd
import tensorflow as tf

Reading Dataset

In [2]:
data=pd.read_csv('/content/mobile_price_classification (1).csv')

In [3]:
data.head()

,battery_power,bluetooth,clock_speed,dual_sim,front_cam,4G,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [16]:
data.shape

(2000, 21)

In [21]:
data['price_range'].unique()

array([1, 2, 3, 0])

Checking null values

In [4]:
data.isnull().sum()

battery_power     0
bluetooth         0
clock_speed       0
dual_sim          0
front_cam         0
4G                0
int_memory        0
m_dep             0
mobile_wt         0
n_cores           0
primary_camera    0
px_height         0
px_width          0
ram               0
sc_h              0
sc_w              0
talk_time         0
three_g           0
touch_screen      0
wifi              0
price_range       0
dtype: int64

Definig x and y

In [5]:
x=data.drop('price_range',axis=1)

In [6]:
y=data['price_range']

In [7]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [8]:
seed=7
tf.random.set_seed(seed)

In [9]:
!pip install scikeras
from scikeras.wrappers import KerasClassifier

Hyper parameter tuning

In [19]:
def create_model(optimizer='SGD'):
 # create model
 model = Sequential()
 model.add(Dense(12, input_shape=(20,), activation='relu'))
 model.add(Dense(1, activation='sigmoid'))
 # Compile model
 model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
 return model
# define the grid search parameters
optimizer = ['SGD',  'Adam', 'Adamax', 'Nadam']
epochs = [10,20,30]
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(model__optimizer=optimizer,optimizer__learning_rate=learn_rate,epochs=epochs)
#Build Model
model = KerasClassifier(build_fn=create_model, verbose=0)

In [20]:
grid = GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3)
grid_result=grid.fit(x,y)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


In [22]:
print('Best %f using %s'%(grid_result.best_score_,grid_result.best_params_))

Best 0.250000 using {'epochs': 10, 'model__optimizer': 'SGD', 'optimizer__learning_rate': 0.001}


Regularisation

In [23]:
from tensorflow.keras.layers import Dropout

In [30]:
def build_model(rate):
  model=Sequential()
  model.add(Dense(8,input_dim=20,activation='relu'))
  model.add(Dropout(rate))

  model.add(Dense(8,input_dim=8,activation='relu'))
  model.add(Dropout(rate))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(loss='binary_crossentropy',optimizer='SGD',metrics=['accuracy'])

  return model



In [31]:
model=KerasClassifier(model=build_model, verbose=0)

In [32]:
rate=[0,0.2,0.4]
epochs=[10,20]
batch_size=[20]

In [33]:
param_grid=dict(model__rate=rate,epochs=epochs,batch_size=batch_size)

In [34]:
grid= GridSearchCV(estimator=model,param_grid=param_grid,cv=3)

In [35]:
grid_result=grid.fit(x,y)

In [36]:
grid_result.best_params_

{'batch_size': 20, 'epochs': 10, 'model__rate': 0}

In [37]:
from tensorflow.keras.regularizers import l2

In [38]:
def build_model(lambda_parameter=0.1):
  model=Sequential()
  model.add(Dense(8,input_dim=20,activation='relu',kernel_regularizer=l2(lambda_parameter)))
  model.add(Dense(8,input_dim=20,activation='relu',kernel_regularizer=l2(lambda_parameter)))
  model.add(Dense(1,input_dim=20,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='SGD',metrics=['accuracy'])
  return model

In [39]:
model=KerasClassifier(model=build_model,verbose=0)

In [40]:
epochs=[10,20]
batch_size=[10,20]
lambda_parameter=[0.1,0.5,1]

In [41]:
param_grid=dict(model__lambda_parameter=lambda_parameter,
                epochs=epochs,batch_size=batch_size)

In [42]:
grid=GridSearchCV(estimator=model,param_grid=param_grid,cv=3)

In [43]:
result=grid.fit(x,y)

In [44]:
result.best_params_

{'batch_size': 10, 'epochs': 10, 'model__lambda_parameter': 0.1}

In [ ]:
result.best_score_

0.6544934640522876